In [2]:

import os
from pydicom import dcmread, dcmwrite
import pydicom
from datetime import datetime


def display_dicom_series_info(dicom_directory):
    """
    Display information about unique DICOM series in the specified directory.

    Args:
        dicom_directory (str): Path to the DICOM directory.

    Returns:
        None
    """
    # Read all DICOM files in the directory
    dicom_files = [dcmread(os.path.join(dicom_directory, f)) for f in os.listdir(dicom_directory)]

    # Dictionary to store series information and count
    series_info = {}

    # Loop through DICOM files and update the dictionary
    for dicom_file in dicom_files:
        series_description = dicom_file.SeriesDescription
        series_number = dicom_file.SeriesNumber
           # Read the DICOM file

        # Extract patient information
        patient_id = dicom_file.PatientID
        birth_date = dicom_file.PatientBirthDate
        patient_name = dicom_file.PatientName

        # Create a unique key for each series
        series_key = (series_description, series_number,patient_id)

        # Update the dictionary
        if series_key in series_info:
            series_info[series_key]["count"] += 1
        else:
            series_info[series_key] = {"description": series_description, "number": series_number, "count": 1,"PATIENT_ID": patient_id,"PATIENT_NAME": patient_name}
     

    grouped_dict = {}
    # Display unique series and count
    for series_key, info in series_info.items():

        key_value = info['PATIENT_ID']
        if key_value not in grouped_dict:
            grouped_dict[key_value] = []
        grouped_dict[key_value].append(info)
    

    # Print grouped objects
    for key_value, grouped_series in grouped_dict.items():
        print(f"DICOMS with ID : '{key_value}':")
        for info in grouped_series:
            print(f"Series Description: {info['description']}, Series Number: {info['number']}, Count: {info['count']}, PATIENT_ID: {info['PATIENT_ID']}, PATIENT_NAME: {info['PATIENT_NAME']}")

    return grouped_dict



def copy_dicom_files_with_filter(dicom_directory,output_dir):
    """
    groups dicoms with same patient id, each in a new folder. 

    Args:
        dicom_directory (str): Path to the DICOM directory.
        output_dir(str): path to output folder for separateed dicoms.

    Returns:
        None
    """

    exclude_series = [
        {"description": "FUJI Basic Text SR for HL7 Radiological Report", "number": 999},
        {"description": "Screen Save", "number": 99},
        {"description": "Dose Report", "number": 999}
    ]
    unique_ids= display_dicom_series_info(dicom_directory)
    for id in unique_ids:
  
        # Read DICOM files with specific patient ID
        dicom_files = [dcmread(os.path.join(dicom_directory, f)) for f in os.listdir(dicom_directory) 
               if dcmread(os.path.join(dicom_directory, f)).PatientID == id]
        

        output_directory = os.path.join(output_dir, id)
        # Create the output directory if it doesn't exist
        os.makedirs(output_directory, exist_ok=True)

        
        # Loop through DICOM files and copy files without the specified series
        for dicom_file in dicom_files:

            series_description = dicom_file.SeriesDescription
            series_number = dicom_file.SeriesNumber

            if any(series_description == excl["description"] or series_number == excl["number"] for excl in exclude_series):
             continue

            dicom_file.PatientID = "ANONYMOUS"

            dicom_file.PatientName = "ANONYMOUS"
            # Anonymize Birthdate
            dicom_file.PatientBirthDate = datetime.today().strftime('%Y%m%d')
            
            # Copy the file to the output directory
            output_file_path = os.path.join(output_directory, os.path.basename(dicom_file.filename))
            # shutil.copy(dicom_file.filename, output_file_path)
            dcmwrite(output_file_path, dicom_file)


        print(f"Output directory: {output_directory}")




In [4]:
files=[
       'D:/Liver Segmentation Meena 2024/Neuroblastoma_2025/34 pd1241369/DICOMOBJ',
       'D:/Liver Segmentation Meena 2024/Neuroblastoma_2025/DICOMOBJ 33 pd1242672 4 recon  3.arterial/DICOMOBJ',
       'D:/Liver Segmentation Meena 2024/Neuroblastoma_2025/DICOMOBJ 39 pd1223300 301 viens/DICOMOBJ 39 pd1223300 301 viens']

for i,file in enumerate(files):
    print(f'file:{i+1}')
    display_dicom_series_info(file)

file:1
DICOMS with ID : '20240221172709108':
Series Description: Dose Report, Series Number: 999, Count: 1, PATIENT_ID: 20240221172709108, PATIENT_NAME: Unknown1^Unknown1
Series Description: 1.25mm ARTERIAL, Series Number: 3, Count: 373, PATIENT_ID: 20240221172709108, PATIENT_NAME: Unknown1^Unknown1
Series Description: FUJI Presentation State - SNAPSHOT, Series Number: None, Count: 1, PATIENT_ID: 20240221172709108, PATIENT_NAME: Unknown1^Unknown1
Series Description: FUJI Basic Text SR for HL7 Radiological Report, Series Number: 999, Count: 1, PATIENT_ID: 20240221172709108, PATIENT_NAME: Unknown1^Unknown1
file:2
DICOMS with ID : '20240221173216345':
Series Description: Dose Report, Series Number: 999, Count: 1, PATIENT_ID: 20240221173216345, PATIENT_NAME: Unknown1^Unknown1
Series Description: Recon 3: Arterial, Series Number: 4, Count: 461, PATIENT_ID: 20240221173216345, PATIENT_NAME: Unknown1^Unknown1
Series Description: Recon 2: Arterial, Series Number: 3, Count: 309, PATIENT_ID: 2024

In [ ]:
files=[
       'D:/Liver Segmentation Meena 2024/Neuroblastoma_2025/34 pd1241369/DICOMOBJ',
       'D:/Liver Segmentation Meena 2024/Neuroblastoma_2025/DICOMOBJ 33 pd1242672 4 recon  3.arterial/DICOMOBJ',
        'D:/Liver Segmentation Meena 2024/Neuroblastoma_2025/DICOMOBJ 35 dm1217752 recon 3/DICOMOBJ',
        'D:/Liver Segmentation Meena 2024/Neuroblastoma_2025/DICOMOBJ 36 pd1232501 thin cuts/DICOMOBJ']

for i,file in enumerate(files):
    print(f'file:{i+1}')
    display_dicom_series_info(file)

file:1
DICOMS with ID : '20240221172709108':
Series Description: Dose Report, Series Number: 999, Count: 1, PATIENT_ID: 20240221172709108, PATIENT_NAME: Unknown1^Unknown1
Series Description: 1.25mm ARTERIAL, Series Number: 3, Count: 373, PATIENT_ID: 20240221172709108, PATIENT_NAME: Unknown1^Unknown1
Series Description: FUJI Presentation State - SNAPSHOT, Series Number: None, Count: 1, PATIENT_ID: 20240221172709108, PATIENT_NAME: Unknown1^Unknown1
Series Description: FUJI Basic Text SR for HL7 Radiological Report, Series Number: 999, Count: 1, PATIENT_ID: 20240221172709108, PATIENT_NAME: Unknown1^Unknown1
file:2
DICOMS with ID : '20240221173216345':
Series Description: Dose Report, Series Number: 999, Count: 1, PATIENT_ID: 20240221173216345, PATIENT_NAME: Unknown1^Unknown1
Series Description: Recon 3: Arterial, Series Number: 4, Count: 461, PATIENT_ID: 20240221173216345, PATIENT_NAME: Unknown1^Unknown1
Series Description: Recon 2: Arterial, Series Number: 3, Count: 309, PATIENT_ID: 2024

In [9]:
import os
import nibabel as nib

def rename_nifti_files(directory, file_extension='.nii.gz'):
    # Get list of files in the directory
    files = [f for f in os.listdir(directory) if f.endswith(file_extension)]
    
    # Sort files (assuming they are sortable by name in numeric order)
    files.sort()
    
    # Loop through the sorted files and rename them
    for i, filename in enumerate(files, start=34):
        old_path = os.path.join(directory, filename)
        new_filename = f'volume_{i}_0000{file_extension}'
        # directory = directory + "/volume"
        new_path = os.path.join(directory, new_filename)
        
        # Rename the file
        os.rename(old_path, new_path)
        print(f'Renamed {filename} to {new_filename}')

# Specify the directory containing the NIfTI files
directory_path = 'D:/Liver Segmentation Meena 2024/Neuroblastoma_2025/volumes/'

# Call the function to rename NIfTI files
rename_nifti_files(directory_path)


Renamed volume_34_0000.nii.gz to volume_34_0000.nii.gz
